In [ ]:
%matplotlib inline


# Bayesian calibration of the flooding model


## Abstract

The goal of this example is to present the statistical hypotheses of the bayesian calibration of the `flooding model<use-case-flood-model>`.



## Parameters to calibrate

The vector of parameters to calibrate is:

\begin{align}\theta = (K_s,Z_v,Z_m).\end{align}


The variables to calibrate are $(K_s,Z_v,Z_m)$ and are set to the following values:

\begin{align}K_s = 30, \qquad Z_v = 50, \qquad Z_m = 55.\end{align}


## Observations

In this section, we describe the statistical model associated with the $n$ observations.
The errors of the water heights are associated with a gaussian distribution with a zero mean and a standard variation equal to:

\begin{align}\sigma=0.1.\end{align}


Therefore, the observed water heights are:

\begin{align}H_i = G(Q_i,K_s,Z_v,Z_m) + \epsilon_i\end{align}


for $i=1,...,n$ where

\begin{align}\epsilon \sim \mathcal{N}(0,\sigma^2)\end{align}


and we make the hypothesis that the observation errors are independent.
We consider a sample size equal to:

\begin{align}n=20.\end{align}


The observations are the couples $\{(Q_i,H_i)\}_{i=1,...,n}$, i.e. each observation is a couple made of the flowrate and the corresponding river height.

## Analysis

In this model, the variables $Z_m$ and $Z_v$ are not identifiables, since only the difference $Z_m-Z_v$ matters. Hence, calibrating this model requires some regularization.



## Generate the observations



In [ ]:
import pylab as pl
from openturns.viewer import View
from openturns.usecases import flood_model as flood_model
import openturns.viewer as viewer
import numpy as np
import openturns as ot
ot.Log.Show(ot.Log.NONE)

A basic implementation of the probabilistic model is available in the usecases module :



In [ ]:
fm = flood_model.FloodModel()

We define the model $g$ which has 4 inputs and one output H.

The nonlinear least squares does not take into account for bounds in the parameters. Therefore, we ensure that the output is computed whatever the inputs. The model fails into two situations:

* if $K_s<0$,
* if $Z_v-Z_m<0$.

In these cases, we return an infinite number.



In [ ]:
def functionFlooding(X):
    L = 5.0e3
    B = 300.0
    Q, K_s, Z_v, Z_m = X
    alpha = (Z_m - Z_v)/L
    if alpha < 0.0 or K_s <= 0.0:
        H = np.inf
    else:
        H = (Q/(K_s*B*np.sqrt(alpha)))**(3.0/5.0)
    return [H]

In [ ]:
g = ot.PythonFunction(4, 1, functionFlooding)
g = ot.MemoizeFunction(g)
g.setOutputDescription(["H (m)"])

We load the input distribution for $Q$.



In [ ]:
Q = fm.Q

Set the parameters to be calibrated.



In [ ]:
K_s = ot.Dirac(30.0)
Z_v = ot.Dirac(50.0)
Z_m = ot.Dirac(55.0)
K_s.setDescription(["Ks (m^(1/3)/s)"])
Z_v.setDescription(["Zv (m)"])
Z_m.setDescription(["Zm (m)"])

We create the joint input distribution.



In [ ]:
inputRandomVector = ot.ComposedDistribution([Q, K_s, Z_v, Z_m])

Create a Monte-Carlo sample of the output H.



In [ ]:
nbobs = 20
inputSample = inputRandomVector.getSample(nbobs)
outputH = g(inputSample)

Generate the observation noise and add it to the output of the model.



In [ ]:
sigmaObservationNoiseH = 0.1  # (m)
noiseH = ot.Normal(0., sigmaObservationNoiseH)
ot.RandomGenerator.SetSeed(0)
sampleNoiseH = noiseH.getSample(nbobs)
Hobs = outputH + sampleNoiseH

Plot the Y observations versus the X observations.



In [ ]:
Qobs = inputSample[:, 0]

In [ ]:
graph = ot.Graph("Observations", "Q (m3/s)", "H (m)", True)
cloud = ot.Cloud(Qobs, Hobs)
graph.add(cloud)
view = viewer.View(graph)

## Setting the calibration parameters



Define the parametric model $z = f(x,\theta)$ that associates each observation $x_i$ and values of the  parameters $\theta_i$ to the parameters of the distribution of the corresponding observation: here $z=(\mu, \sigma)$



In [ ]:
def fullModelPy(X):
    Q, K_s, Z_v, Z_m = X
    H = g(X)[0]
    sigmaH = 0.5  # (m^2) The standard deviation of the observation error.
    return [H, sigmaH]


fullModel = ot.PythonFunction(4, 2, fullModelPy)
model = ot.ParametricFunction(fullModel, [0], Qobs[0])
model

Define the value of the reference values of the $\theta$ parameter. In the bayesian framework, this is called the mean of the *prior* gaussian distribution. In the data assimilation framework, this is called the *background*.



In [ ]:
KsInitial = 20.
ZvInitial = 49.
ZmInitial = 51.
parameterPriorMean = [KsInitial, ZvInitial, ZmInitial]
paramDim = len(parameterPriorMean)

Define the covariance matrix of the parameters $\theta$ to calibrate.



In [ ]:
sigmaKs = 5.
sigmaZv = 1.
sigmaZm = 1.

In [ ]:
parameterPriorCovariance = ot.CovarianceMatrix(paramDim)
parameterPriorCovariance[0, 0] = sigmaKs**2
parameterPriorCovariance[1, 1] = sigmaZv**2
parameterPriorCovariance[2, 2] = sigmaZm**2
parameterPriorCovariance

Define the the prior distribution $\pi(\underline{\theta})$ of the parameter $\underline{\theta}$



In [ ]:
prior = ot.Normal(parameterPriorMean, parameterPriorCovariance)
prior.setDescription(['Ks', 'Zv', 'Zm'])
prior

Define the distribution of observations $\underline{y} | \underline{z}$ conditional on model predictions.

Note that its parameter dimension is the one of $\underline{z}$, so the model must be adjusted accordingly. In other words, the input argument of the `setParameter` method of the conditional distribution must be equal to the dimension of the output of the `model`. Hence, we do not have to set the actual parameters: only the type of distribution is used.



In [ ]:
conditional = ot.Normal()
conditional

Proposal distribution: uniform.



In [ ]:
proposal = [ot.Uniform(-5., 5.), ot.Uniform(-1., 1.), ot.Uniform(-1., 1.)]
proposal

## Test the MCMC sampler

The MCMC sampler essentially computes the log-likelihood of the parameters.



In [ ]:
mymcmc = ot.MCMC(prior, conditional, model, Qobs, Hobs, parameterPriorMean)

In [ ]:
mymcmc.computeLogLikelihood(parameterPriorMean)

## Test the Metropolis-Hastings sampler



- Creation of the Random Walk Metropolis-Hastings (RWMH) sampler.



In [ ]:
initialState = parameterPriorMean

In [ ]:
RWMHsampler = ot.RandomWalkMetropolisHastings(
    prior, conditional, model, Qobs, Hobs, initialState, proposal)

Tuning of the RWMH algorithm.



Strategy of calibration for the random walk (trivial example: default).



In [ ]:
strategy = ot.CalibrationStrategyCollection(paramDim)
RWMHsampler.setCalibrationStrategyPerComponent(strategy)

Other parameters.



In [ ]:
RWMHsampler.setVerbose(True)
RWMHsampler.setThinning(1)
RWMHsampler.setBurnIn(200)

Generate a sample from the posterior distribution of the parameters theta.



In [ ]:
sampleSize = 1000
sample = RWMHsampler.getSample(sampleSize)

Look at the acceptance rate (basic checking of the efficiency of the tuning; value close to 0.2 usually recommended).



In [ ]:
RWMHsampler.getAcceptanceRate()

Build the distribution of the posterior by kernel smoothing.



In [ ]:
kernel = ot.KernelSmoothing()
posterior = kernel.build(sample)

Display prior vs posterior for each parameter.



In [ ]:
fig = pl.figure(figsize=(12, 4))

for parameter_index in range(paramDim):
    graph = posterior.getMarginal(parameter_index).drawPDF()
    priorGraph = prior.getMarginal(parameter_index).drawPDF()
    priorGraph.setColors(['blue'])
    graph.add(priorGraph)
    graph.setLegends(['Posterior', 'Prior'])
    ax = fig.add_subplot(1, paramDim, parameter_index+1)
    _ = ot.viewer.View(graph, figure=fig, axes=[ax])

_ = fig.suptitle("Bayesian calibration")